In [1]:
# The BN254 Elliptic curve E(Fp) : y^2 = x^3 + 3 
p = 21888242871839275222246405745257275088696311157297823662689037894645226208583
Fp = GF(p)
E = EllipticCurve(Fp, [0,3])
r = E.order()
Fr = GF(r)

# The polynomial Ring over Fr
R.<x> = PolynomialRing(Fr)

# Generates a random polynomial of degree less than n
def rand_poly(n):
    return R([R.base_ring().random_element() for _ in range(n)])

In [2]:
# Generates the bases
def setup(n): 
    G = [E.random_element() for i in range(n)]
    H = E.random_element()
    return G, H

# Commits to a polynomial
def commit(f, G, H):
    assert f.degree() < len(G)
    # Generates a random blinding factor bf
    bf = Fr.random_element()
    Cf = sum([ai*Gi for ai, Gi in zip(f.coefficients(),G)]) + bf*H
    return Cf, bf

# Executes an interactive protocol showing that a claimed f(z) is correct with respect to a commitment to f
def prove_verify_ip(f, z, Cf, bf, n, G, H):
    # The prover computes the evaluation f(z) and sends it to the Verifier
    c = f(z)

    # The Verifier generates a random generator U of E and sends it to the Prover
    U = E.random_element()

    # The Prover and the Verifier initialize G, P
    G = G
    P = Cf
    
    # The Prover initialize also A, B, b
    A = f.coefficients() + [Fr(0)]*(n-f.degree()-1)
    B = [z^i for i in range(n)]
    b = bf

    # Prover and Verifier iteratively Split and Fold the input
    while len(G) != 1:
        
        n = len(G)
        assert n == len(A) == len(B)

        # Prover and Verifier split the vectors G
        G_lo, G_hi = G[:n/2], G[n/2:]

        # The Prover splits also the vectors A, B
        A_lo, A_hi = A[:n/2], A[n/2:]
        B_lo, B_hi = B[:n/2], B[n/2:]

        # The Prover computes partial commitments L,R from random challenges l,r and sends them to the Verifier
        λ = Fr.random_element()
        ρ = Fr.random_element()
        L = λ*H + sum([a_l*g_h for a_l,g_h in zip(A_lo, G_hi)]) + sum([a_l*b_h for a_l,b_h in zip(A_lo, B_hi)])*U
        R = ρ*H + sum([a_h*g_l for a_h,g_l in zip(A_hi, G_lo)]) + sum([a_h*b_l for a_h,b_l in zip(A_hi, B_lo)])*U
        
        # The Verifier picks a random round challenge x and sends it to the Prover
        α = Fr.random_element()
    
        # Prover and Verifier folds at each round G and updates P
        G = [α^-1*g_l + α*g_h for g_l,g_h in zip(G_lo, G_hi)]
        P = α^2*L + P + α^-2*R
    
        # The Prover folds A and B and updates b
        A = [α*a_l + α^-1*a_h for a_l,a_h in zip(A_lo, A_hi)]
        B = [α^-1*b_l + α*b_h for b_l,b_h in zip(B_lo, B_hi)]
        b = α^2*λ + b + α^-2*ρ

    # At this point n = 1 and the Prover sends to Verifier the values A[0], B[0], b 
    # The Verifier ends the protocol by checking if the following equation holds:
    return P + c*U == A[0]*G[0] + b*H + A[0]*B[0]*U

In [3]:
# The degree bound
k = 8
n = 1 << k

# Setup
G, H = setup(n)

# We generate a random polynomial
f = rand_poly(n)

# The Prover commits to f
Cf, bf = commit(f, G, H)

# The Verifier picks a random evaluation point z and sends it to the Prover
z = Fr.random_element()

# The Prover and the Verifier engage in an interactive protocol showing that the claimed f(z) is correct with respect to Cf
assert prove_verify_ip(f, z, Cf, bf, n, G, H)